In [5]:
import pandas as pd
import requests
import time
import getpass
import os
'''
client_id = 'Nrbc6n5KVQ6fEQ_7JUfH3A'
client_secret = 'Y0STrnfxdJh_r5h9TS5qxx2tAs5swg'#alphanumeric string provided as "secret"
user_agent = 'data' #the name of your application
username =  'Important_Trade_7759'#your reddit username
password = 'saima123'
'''
client_id = getpass.getpass()#alphanumeric string provided under "personal use script"
client_secret = getpass.getpass() #alphanumeric string provided as "secret"
user_agent = getpass.getpass() #the name of your application
username =  getpass.getpass()#your reddit username
password =  getpass.getpass()#your reddit password'''

auth = requests.auth.HTTPBasicAuth(client_id, client_secret)
data = {
    'grant_type': 'password',
    'username': username,
    'password': password
}

#create an informative header for your application
headers = {'User-Agent': 'subreddit_data1/0.0.1'}
res = requests.post(
    'https://www.reddit.com/api/v1/access_token',
    auth=auth,
    data=data,
    headers=headers)

print(res)
res.json()

#retrieve access token
token = res.json()['access_token']

headers['Authorization'] = f'bearer {token}'

requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).status_code == 200
# Generalized function to fetch posts from any category
def fetch_posts(subreddit, category, after=None, limit=100):
    url = f"https://www.reddit.com/r/{subreddit}/{category}.json"  # Dynamic category
    headers = {'User-Agent': 'your_user_agent'}
    params = {
        'limit': limit,  # Max 100 posts per request
        'after': after   # Pagination token
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch posts: {response.status_code}")
        return None

# Function to get unique posts and return as DataFrame
def get_unique_posts_dataframe(subreddit, category):
    '''
    '''
    posts_data = []
    unique_ids = set()  # Set to store unique post IDs
    max_posts = 1000  # Limit for fetching
    total_posts = 0
    after = None

    while total_posts < max_posts:
        data = fetch_posts(subreddit, category, after)
        if data is None:
            break  # Stop fetching if there's an error

        posts = data['data']['children']

        # Extract title, assign subreddit, and category, ensuring uniqueness by ID
        for post in posts:
            post_id = post['data']['id']
            if post_id not in unique_ids:
                unique_ids.add(post_id)
                posts_data.append({
                    'title': post['data']['title'],
                    'subreddit': subreddit,
                    'category': category
                })
                total_posts += 1

            if total_posts >= max_posts:
                break

        after = data['data']['after']  # Get the next "after" token
        if after is None:
            break  # No more posts to fetch

        time.sleep(1)  # Avoid hitting Reddit API rate limits

    # Convert collected posts to DataFrame
    return pd.DataFrame(posts_data)

# Ensure data directory exists
if not os.path.exists('./data'):
    os.makedirs('./data')

# General function to fetch posts from subreddits and categories
def fetch_and_save_posts(subreddit, categories, filename):
    '''
    '''
    df_list = []
    
    for category in categories:
        df_posts = get_unique_posts_dataframe(subreddit, category)
        df_list.append(df_posts)
    
    df_combined = pd.concat(df_list, ignore_index=True)

    # Save to CSV, if file doesn't exist, create it; if it exists, append
    if not os.path.exists(filename):
        df_combined.to_csv(filename, index=False)
    else:
        df_combined.to_csv(filename, mode='a', header=False, index=False)

# Fetch posts from "gadgets" and "technology" subreddits
subreddit_list = ['gadgets', 'technology']
categories = ['new']

for subreddit in subreddit_list:
    fetch_and_save_posts(subreddit, categories, './data/dataframe.csv')

# Display the final shape of the DataFrame
df = pd.read_csv('./data/dataframe.csv')
print(f"Total posts collected: {df.shape[0]}")



 ········
 ········
 ········
 ········
 ········


<Response [200]>
Total posts collected: 3318


In [ ]:
<Response [200]>
Total posts collected: 1657